# Installs & imports

In [1]:
import os
import re
from collections import defaultdict
import pandas as pd
import torch
import random
import numpy as np
import json

from transformers import (
    AutoTokenizer, 
    RobertaTokenizerFast, 
    DataCollatorForTokenClassification,
    AutoModelForTokenClassification, 
    TrainingArguments, 
    Trainer
)

from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset

import nltk
nltk.download('punkt')

import wandb
wandb.login()

/opt/conda/lib/python3.8/site-packages/apex/pyprof/__init__.py:5: FutureWarning: pyprof will be removed by the end of June, 2022
  warnings.warn("pyprof will be removed by the end of June, 2022", FutureWarning)
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
wandb: Currently logged in as: julia_kor. Use `wandb login --relogin` to force relogin


True

In [2]:
%env WANDB_PROJECT=Sentiment_Aspect

env: WANDB_PROJECT=Sentiment_Aspect


In [3]:
import random

def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

In [4]:
RANDOM_SEED = 1234
set_random_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Train

In [6]:
df_train = pd.read_csv('data/aspects_train.csv', index_col=0)
df_val = pd.read_csv('data/aspects_val.csv', index_col=0)

In [7]:
MODEL_NAME = 'sberbank-ai/ruRoberta-large'

In [8]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, add_prefix_space=True)

In [9]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [10]:
model = AutoModelForTokenClassification.from_pretrained(MODEL_NAME, num_labels=4)

/opt/conda/lib/python3.8/site-packages/torch/nn/modules/module.py:1402: UserWarning: positional arguments and argument "destination" are deprecated. nn.Module.state_dict will not accept them in the future. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(
Some weights of the model checkpoint at sberbank-ai/ruRoberta-large were not used when initializing RobertaForTokenClassification: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you 

In [11]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2).reshape((-1,))
    labels = labels.reshape((-1,))

    true_predictions = [p for (p, l) in zip(predictions, labels) if l != -100]
    true_labels = [l for (p, l) in zip(predictions, labels) if l != -100]

    accuracy = accuracy_score(true_labels, true_predictions)
    return {
        "accuracy": accuracy,
    }

In [12]:
label_all_tokens = True

class CustomDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.class_to_int = {"positive": 0, "negative": 1, "neutral": 2, "both": 3}

    def __len__(self):
        return len(self.dataset["tokens"])

    def __getitem__(self, idx):
        tokenized_inputs = tokenizer(self.dataset["tokens"].iloc[idx], truncation=True, is_split_into_words=True, max_length=512)
        label = self.dataset["sentiment"].iloc[idx]
        cls = self.dataset["class"].iloc[idx]
        word_ids = tokenized_inputs.word_ids()
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None or cls[word_idx] == "O":
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(self.class_to_int[label[word_idx]])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(self.class_to_int[label[word_idx]] if label_all_tokens else -100)
            previous_word_idx = word_idx
        tokenized_inputs["labels"] = label_ids
        return tokenized_inputs

In [13]:
df_train = df_train.groupby("idx").agg(list)
df_val = df_val.groupby("idx").agg(list)

In [14]:
ds_train = CustomDataset(df_train)
ds_dev = CustomDataset(df_val)

In [15]:
for name,param in model.named_parameters():
    if not re.search("classifier|23|22|21|20", name):
        param.requires_grad = False

In [16]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    report_to="wandb",
    
    run_name="aspects_sent/experiment_2",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=10,
    learning_rate=3e-5,
    weight_decay=0.1,
    warmup_ratio=0.06,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_train,
    eval_dataset=ds_dev,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 227
  Num Epochs = 10
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 290
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


/opt/conda/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
1,1.109900,0.796730,0.731065
2,1.034800,0.772404,0.737651
3,0.838400,0.718836,0.718990
4,0.749700,0.692544,0.705818
5,0.617700,0.617653,0.748628
6,0.631900,0.555931,0.773875
7,0.638900,0.591757,0.773875
8,0.636800,0.571052,0.751921
9,0.540100,0.571697,0.772777
10,0.585300,0.552190,0.772777


***** Running Evaluation *****
  Num examples = 28
  Batch size = 8
/opt/conda/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 28
  Batch size = 8
/opt/conda/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 28
  Batch size = 8
/opt/conda/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along 

TrainOutput(global_step=290, training_loss=0.7212927785413018, metrics={'train_runtime': 1102.1541, 'train_samples_per_second': 2.06, 'train_steps_per_second': 0.263, 'total_flos': 1208290453905864.0, 'train_loss': 0.7212927785413018, 'epoch': 10.0})

In [17]:
torch.save(model, "model_aspect_sent.pt")